In [1]:
import numpy as np
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.style.use("seaborn")

/home/jcartus/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Fetch dataset


In [2]:
from SCFInitialGuess.utilities.dataset import make_butadien_dataset


dataset, molecules = make_butadien_dataset(
    np.load("molecules_platin_lanl2dz.npy"),
    *np.load("dataset_platin_lanl2dz.npy")
)

[-] 2018-05-23 16:13:11: Data set normalized. Mean value std: 0.007695333664764158


# Extract diagonal 

In [3]:
from SCFInitialGuess.utilities.dataset import extract_triu

def extract_diagonal(x, dim=88):
    x_batch = x.reshape(-1, dim, dim)
    return np.array(list(map(np.diag, x_batch))).reshape(-1, 88)

def extract_all_trius(x, dim=88):
    return np.array(list(map(lambda y: extract_triu(y, dim), x)))

dataset.training = (extract_all_trius(dataset.training[0]), extract_diagonal(dataset.training[1]))
dataset.validation = (extract_all_trius(dataset.validation[0]), extract_diagonal(dataset.validation[1]))
dataset.testing = (extract_all_trius(dataset.testing[0]), extract_diagonal(dataset.testing[1]))

# Train a network

In [ ]:
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.cost_functions import RegularizedMSE, MSE
from SCFInitialGuess.nn.training import Trainer

dim = 88

trainer = Trainer(
    EluTrNNN([int(dim * (dim + 1) / 2), 2000, 1000, 500, 200, 150, dim]),
    cost_function=MSE(),#RegularizedMSE(alpha=1e-7),
    optimizer=tf.train.AdamOptimizer(learning_rate=5e-3)
)

trainer.setup()

network, sess = trainer.train(
    dataset,
    convergence_threshold=5e-7
)
    

[-] 2018-05-23 16:14:17: No target graph specified for Trainer setup. Creating new graph ...
[-] 2018-05-23 16:14:17: Setting up the training in the target graph ...
[-] 2018-05-23 16:14:17: network ...
[-] 2018-05-23 16:14:17: error function ...
[-] 2018-05-23 16:14:17: cost function ...
[-] 2018-05-23 16:14:17: training step
[-] 2018-05-23 16:14:18: Starting network training ...
[ ] 2018-05-23 16:14:18: Val. Cost: 8.798E+03. Error: 8.798E+03. Diff: 1.0E+10
[ ] 2018-05-23 16:15:10: Val. Cost: 1.787E+02. Error: 1.787E+02. Diff: 8.6E+03
[ ] 2018-05-23 16:16:01: Val. Cost: 3.464E+01. Error: 3.464E+01. Diff: 1.4E+02
[ ] 2018-05-23 16:16:54: Val. Cost: 3.269E+01. Error: 3.269E+01. Diff: 1.9E+00
[ ] 2018-05-23 16:17:46: Val. Cost: 2.302E+01. Error: 2.302E+01. Diff: 9.7E+00
[ ] 2018-05-23 16:18:40: Val. Cost: 1.501E+01. Error: 1.501E+01. Diff: 8.0E+00
[ ] 2018-05-23 16:19:33: Val. Cost: 1.312E+01. Error: 1.312E+01. Diff: 1.9E+00
[ ] 2018-05-23 16:20:27: Val. Cost: 8.460E+00. Error: 8.460E+00